In [1]:
import json
from tqdm import tqdm


In [11]:
train_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_train.txt'
dev_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_dev.txt'
test_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_test.txt'
train_original_in = '/mnt/scratch/chenqu/msmarco_wasi/original/session_train_original.txt'
all_data_in = '/mnt/scratch/chenqu/msmarco_wasi/original/all_data.txt'

dev_out = '/mnt/scratch/chenqu/msmarco/original/session_dev.txt'
test_out = '/mnt/scratch/chenqu/msmarco/original/session_test.txt'

In [3]:
train_query_dict = {}
with open(train_in) as fin:
    for line in fin:
        session = json.loads(line.strip())
        for turn_id, turn in enumerate(session['query']):
            query = turn['text']
            train_query_dict[query] = 1

In [14]:
def filter_dev(dev_in):
    lines_not_in_train = []
    with open(dev_in) as fin:
        for line in tqdm(fin):
            keep = True
            session = json.loads(line.strip())
            for turn_id, turn in enumerate(session['query']):
                query = turn['text']
                if query in train_query_dict:
                    keep = False
                    break
            if keep:
                lines_not_in_train.append(line)
    return lines_not_in_train

In [15]:
lines_not_in_train = filter_dev(all_data_in)

276381it [01:19, 3465.03it/s]


In [16]:
len(lines_not_in_train)

546